In [ ]:
from pybis import Openbis

import os
from decouple import config as environ
import time

In [ ]:
o = Openbis(environ("OPENBIS_URL"))
o.login(
    environ("OPENBIS_USERNAME"),
    environ("OPENBIS_PASSWORD"),
    save_token=True,
)

In [ ]:
space = o.get_space(environ("OPENBIS_SPACE"))
project_name = "Zeitmessung"
if project_name.upper() in [p.code for p in space.get_projects()]:
        project = space.get_project("Zeitmessung")
else:
        project = space.new_project("Zeitmessung")
        project.save()



In [ ]:
collection_name="Normal"
if collection_name.upper() in [c.code for c in project.get_collections()]:
    collection_openbis_n = space.get_collection(
        f"/{environ("OPENBIS_SPACE")}/{project_name}/{collection_name}".upper()
    )
else:
    collection_openbis_n = o.new_collection(
        code=collection_name.replace(" ", "_").upper(),
        type="COLLECTION",
        project=project,
    )
collection_openbis_n.save()

In [ ]:
collection_name="Transaction"
if collection_name.upper() in [c.code for c in project.get_collections()]:
    collection_openbis_t = space.get_collection(
        f"/{environ("OPENBIS_SPACE")}/{project_name}/{collection_name}".upper()
    )
else:
    collection_openbis_t = o.new_collection(
        code=collection_name.replace(" ", "_").upper(),
        type="COLLECTION",
        project=project,
    )
collection_openbis_t.save()

In [ ]:
import matplotlib.pyplot as plt

anzahlen = [1, 10, 50, 100]

zeiten_normal = []
zeiten_transaction = []

for anzahl in anzahlen:

    # ------------------------
    # Without Transaction
    # ------------------------
    start_time = time.time()

    for i in range(anzahl):
        obj = o.new_object(
            type="EXPERIMENTAL_STEP",
            code=f"{os.urandom(6).hex()}Test{i}",
            space=environ("OPENBIS_SPACE"),
            project="Zeitmessung",
            collection=collection_openbis_n,
            props={"$name": f"Zeittest: {i}"},
        )
        obj.save()

    end_time = time.time()
    zeiten_normal.append(end_time - start_time)

    # ------------------------
    # With Transaction
    # ------------------------
    start_time = time.time()

    transaction = o.new_transaction()

    for i in range(anzahl):
        obj = o.new_object(
            type="EXPERIMENTAL_STEP",
            code=f"{os.urandom(6).hex()}Test{i}",
            space=environ("OPENBIS_SPACE"),
            project="Zeitmessung",
            collection=collection_openbis_t,
            props={"$name": f"Zeittest: {i}"},
        )
        transaction.add(obj)

    transaction.commit()

    end_time = time.time()
    zeiten_transaction.append(end_time - start_time)

print("Zeiten ohne Transaktion:", zeiten_normal)
print("Zeiten mit Transaktion:", zeiten_transaction)
# ------------------------
# Plot
# ------------------------
plt.figure()
plt.plot(anzahlen, zeiten_normal, marker="o")
plt.plot(anzahlen, zeiten_transaction, marker="o")

plt.xlabel("Anzahl der Objekte")
plt.ylabel("Zeit (Sekunden)")
plt.title("Zeitmessung: Normal vs Transaction")
plt.legend(["Normal", "Transaction"])
plt.xscale("log")  # optional sinnvoll bei 1–1000
plt.show()